In [ ]:
import os
from dotenv import load_dotenv
from typing import TypedDict

load_dotenv()


True

In [ ]:
from langchain_groq import ChatGroq
from langchain_deepseek import ChatDeepSeek
from langgraph.graph import StateGraph, START, END
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser


In [3]:
LLMS = {}

LLMS['gemini'] = ChatGoogleGenerativeAI(
    api_key = os.getenv("GOOGLE_API_KEY"),
    model = 'gemini-1.5-flash'
)

LLMS['deepseek'] = ChatDeepSeek(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    api_base=os.getenv("OPENROUTER_BASE_URL"),
    model="deepseek/deepseek-chat-v3-0324:free"
)

LLMS['moonshotai'] = ChatGroq(
    api_key = os.getenv("GROQ_API_KEY"),
    model = "moonshotai/kimi-k2-instruct"
)


In [ ]:
llm = LLMS['deepseek']


In [ ]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str


In [ ]:
outline_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are an expert blog strategist.
    Given a blog topic, your job is to create a clear, engaging outline that a smart human writer could confidently use to write a great article.

    Keep these in mind:

    - Start with a working title (can be improved later).
    - Break the blog down into sections with informative headings.
    - Under each heading, add brief bullet points on what should be covered.
    - Structure should have a clear intro, a logical flow of main sections, and a strong wrap-up or conclusion.
    - Use natural language. No corporate buzzwords, no generic filler like “content is king.”
    - Don't force trends or clichés unless they genuinely fit.
    - Aim for clarity, usefulness, and a bit of personality.
    - Optionally, suggest a few subtopics, angles, or FAQs if helpful.

    Your tone should feel like a thoughtful, experienced content editor helping a writer get started.
    Make it feel intentional, not autogenerated.
    """),
    ("human", "Generate an outline for this title: {title}")
])


In [ ]:
blog_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are a skilled, thoughtful blog writer.
    You're given a topic and an outline—your job is to turn them into a complete, engaging blog post that reads like it was written by a smart, confident human.

    Follow these principles:

    - Stick to the outline, but feel free to improve flow or connect sections smoothly.
    - Write with a clear, natural voice. Imagine explaining the topic to a smart peer—not oversimplified, not overly formal.
    - Avoid robotic phrases, buzzwords, and filler like:
        “In today's digital age”
        “Leveraging synergies”
        “Cutting-edge solutions”
    - No padding. No corporate fluff. Every paragraph should have purpose.
    - Use conversational transitions where it helps (e.g., “Here's the thing,” “Let's break it down,” “What this really means is…”).
    - Vary sentence structure and rhythm like a real human would.
    - If helpful, use light humor, metaphors, or relatable examples—but don't overdo it.
    - Where appropriate, use emojis sparingly to make it feel friendly 😊
    - End with a clear, satisfying conclusion—either a takeaway, call to action, or reflection.

    Tone: Clear, confident, real. Not trying to impress. Just trying to be helpful and readable.
    """),
    ("human", """
        Generate the blog for the below given topic:
        {title}.
        Here is the outline for this:
        {outline}
    """)
])


In [ ]:
def create_outline(state: BlogState) -> BlogState:
    title = state['title']
    outline_chain = outline_prompt | llm | StrOutputParser()
    outline = outline_chain.invoke(input={
        'title': title,
    })
    state['outline'] = outline
    
    return state
    

In [ ]:
def create_blog(state: BlogState) -> BlogState:
    title = state['title']
    outline = state['outline']
    blog_chain = blog_prompt | llm | StrOutputParser()
    content = blog_chain.invoke(input={
        'title': title,
        'outline': outline
    })
    state['content'] = content
    
    return state


In [ ]:
graph = StateGraph(BlogState)

graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

workflow = graph.compile()


In [ ]:
initial_state = {
    'title': "How to build habits that don't fall apart after 3 days"
}

final_state = workflow.invoke(initial_state)
# print(final_state)


In [ ]:
print(final_state['title'])
print("*"*100)
print(final_state['outline'])
print("*"*100)
print(final_state['content'])


How to build habits that don't fall apart after 3 days
****************************************************************************************************
**Title:** *How to Build Habits That Don’t Fall Apart After 3 Days*  

**Intro:**  
- Open with a relatable anecdote or stat about how most people fail at new habits within days—but why that’s not the whole story.  
- Emphasize that the problem isn’t willpower; it’s strategy.  
- Preview the actionable steps ahead.  

### **1. Start Smaller Than You Think**  
- Why tiny habits work: The brain resists big changes, not small ones.  
- Example: "Floss one tooth" vs. "Floss all your teeth."  
- How to scale down any goal to something laughably easy—and why that builds momentum.  

### **2. Attach New Habits to Existing Ones (Habit Stacking)**  
- The science of cue-routine-reward loops.  
- Formula: "After [current habit], I’ll do [new habit]."  
- Example: "After I pour my morning coffee, I’ll write one sentence in my journal."  
- Com